# 爬取台大招生網之榜單

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import certifi

headers = {'User-Agent': 'User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}

title = None
while title == None:
    response = requests.get("https://www.aca.ntu.edu.tw/w/aca/LocalAdmissionClass_21072014071675661", headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    title = soup.find('a', text='113碩士班招生第2梯次錄取名單')
    if title == None:
        time.sleep(10)
    else:
        break

url = title["href"]        

# 下載指定年份之榜單PDF

In [ ]:
import os

# 指定資料夾
folder_path = "/Users/RYAN/OneDrive/桌面/Python/自練/自動查榜機器人"

# 下載連結內容
response = requests.get(url)

# 初始文件名
filename = os.path.basename(url)

# 将檔案存到資料夾中
with open(os.path.join(folder_path, '榜單.pdf'), "wb") as file:
    file.write(response.content)

# 讀取已下載之PDF

In [ ]:
from PyPDF2 import PdfReader
a = ''
# 打開PDF文件
with open('榜單.pdf', 'rb') as file:
    # 使用PdfReader讀取文件
    pdf = PdfReader(file)

    # PDF頁數
    num_pages = len(pdf.pages)

    # 將內文轉到python，並以字串存著
    for page in pdf.pages:
        a += page.extract_text()

In [ ]:
a # 爬取結果檢查

'1 \n 國立臺灣大學 113學年度碩士班招生  \n       【第2梯次放榜 】正、備取生名單  113 年 3 月 22 日 \n備註一：榜單內容如有爭議，以本校招生委員會之正式公告為準。  \n備註二：本梯次【 第2梯次放榜 】之正取（備取）生請務必詳閱報到各相關要點，如期\n辦理報到， 以免逾期致被取消入學（遞補）資格。  \n一、 網路報到時間： 113年3月26日10:00 至3月28日24:00止。 \n請於報到時間內依規定至「碩士班招生考試報到系統」辦理網路報到，未依\n規定辦理或逾期未報到者， 正取生取消其入學資格論，備取生取消其遞補資\n格，事後不得以 任何理由要求補報到。  \n二、 報到程序：  \n1. 有口試系所組 學位學程 ：於報到時間內至「碩士班招生考試報到系統」\n辦理網路報到即可，但 辦理新生註冊時 須持學力證明及身分證 繳驗。 \n2.網路報到時須依系統指示上傳個人數位照片電子檔，規格如下： 最近6\n個月內2吋正面脫帽半身照，為 1.5英吋寬 × 2.0英吋高之灰階黑白\n或高階彩色影像；影像寬×高像素不得少於 450×600 pixels ；掃描解析\n度300~500 dpi ，檔案以 JPEG格式儲存 (副檔名JPG)。 \n三、 正、備取考生報考資格經審查不符者，取消其入學 (備取)資格，已辦理報到\n且通過資 格審核之備取生，在同系所組 學位學程 且同身分別之所缺名額內依\n備取名次順序登記遞補，遞補至該系所組 學位學程 原核定招生名額為止 ；錄\n取考生若 註冊時無法繳驗畢業證書正本 ，可用其他學歷證明文件 (例如學位\n證明書)替代，但無法繳驗學歷證明文件 正本者，屆時取消其入學資格。  \n四、 考生如同時錄取同年度甄試及本項考試，需於報到時切結「擇一系所組 學位\n學程註冊就讀」 ，未切結者不得辦理報到，如同時錄取正、備取多系所組 學\n位學程時，可於網路報到時依個人意願登記分發志願順序，分發志願順序一\n經填妥送出後，考生不得以任何理由要求更改志願及順序。 本校於報到截止\n後依各系所組 學位學程 缺額情形辦理備取生遞補分發，考生如同獲多所遞補\n資格時，本校將依其志願順序擇 1系所組學位學程 遞補錄取。  \n五、 簡章中規定可選填志願之系所 學位學程 ，若該系所 學位學程 之分組中於報到\n後有缺

# 抓出台大會研所的榜單

In [ ]:
import re
start = re.search('703',a).span()[0]
end = re.search('18名',a).span()[1]

distract =  a[start:end]

In [ ]:
def split_string_in_half(input_string):
    # 計算字符串的長度
    length = len(distract)
    
    # 確定分隔位置
    middle_index = length // 2
    
    # 使用切片操作將字符串分成兩半
    first_half = distract[:middle_index+30]
    second_half = distract[middle_index+30:]
    
    return first_half, second_half

distract1, distract2 = split_string_in_half(distract)

# LINE通知

In [ ]:
import requests

url = 'https://notify-api.line.me/api/notify'
token = '4reHLkJ4Dt2uHl4qIlsHcB5jmK6w159xQfj1qc1jNzk'  # 群組
headers = {
    'Authorization': 'Bearer ' + token    # 設定權杖
}

data = {
    'message': distract1  # 設定要發送的訊息
}

data = requests.post(url, headers=headers, data=data)   # 使用 POST 方法


data = {
    'message': distract2  # 設定要發送的訊息
}

data = requests.post(url, headers=headers, data=data)   # 使用 POST 方法

# 必要時可以郵件通知